# Day 3: 데이터 통합 및 파생변수 생성 (Data Integration)

**날짜**: 2025-07-06

**목표**:
- Day 2에서 정제한 3개 데이터프레임 로드
- 자치구 기준으로 데이터 병합 (Inner Join)
- 분석에 필요한 파생변수 생성
  - 인구당 CCTV 지표
  - 인구당 범죄율 지표
  - 범죄 유형별 비율
- 통합 데이터 검증 및 저장

**활용 도구**:
- utils 모듈의 개선된 함수들 적극 활용

In [ ]:
# 라이브러리 임포트
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from utils.constants import (
    SEOUL_DISTRICTS, CCTV_RANGE, CRIME_RANGE, CCTV_EFFECT_CRIMES,
    RANDOM_SEED, DATA_PATHS, ANALYSIS_YEAR
)
from utils.helpers import (
    set_korean_font, set_plot_style, print_data_info, save_csv_safely,
    validate_data, create_summary_stats, check_district_consistency,
    calculate_ratio_columns
)

warnings.filterwarnings('ignore')

# 환경 설정
set_korean_font()
set_plot_style()
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("\n✅ 모든 라이브러리 임포트 완료")
print(f"📊 분석 연도: {ANALYSIS_YEAR}")
print(f"🎯 Day 3: 데이터 통합 및 파생변수 생성\n")

## 1. Day 2에서 정제한 데이터 로드

In [ ]:
# 정제된 데이터 파일 경로
cctv_path = os.path.join(DATA_PATHS['processed'], 'cctv_cleaned.csv')
crime_path = os.path.join(DATA_PATHS['processed'], 'crime_cleaned.csv')
population_path = os.path.join(DATA_PATHS['processed'], 'population_cleaned.csv')

# 데이터 로드
print("📂 정제된 데이터 로드 중...\n")
try:
    cctv_df = pd.read_csv(cctv_path, encoding='utf-8-sig')
    crime_df = pd.read_csv(crime_path, encoding='utf-8-sig')
    population_df = pd.read_csv(population_path, encoding='utf-8-sig')
    print("✅ 모든 데이터 파일 로드 성공\n")
except FileNotFoundError as e:
    print(f"❌ 파일을 찾을 수 없습니다: {e}")
    print("⚠️ Day 2 노트북을 먼저 실행하세요.")
    raise

# 데이터 크기 확인
print(f"CCTV: {cctv_df.shape}")
print(f"범죄: {crime_df.shape}")
print(f"인구: {population_df.shape}")

In [ ]:
# 자치구명 일치 여부 재확인
consistency_result = check_district_consistency(
    cctv_df, crime_df, population_df,
    district_col='자치구'
)

if not consistency_result['consistent']:
    print("⚠️ 자치구명 불일치 발견. 병합 전 확인 필요.")
    raise ValueError("자치구명 불일치")

## 2. 데이터 병합 (Inner Join)

In [ ]:
# 3개 데이터프레임 병합 (자치구 기준)
print("🔗 데이터 병합 중...\n")

# Step 1: CCTV + 범죄
merged_df = pd.merge(
    cctv_df,
    crime_df,
    on='자치구',
    how='inner'
)
print(f"Step 1 (CCTV + 범죄): {merged_df.shape}")

# Step 2: + 인구
merged_df = pd.merge(
    merged_df,
    population_df,
    on='자치구',
    how='inner'
)
print(f"Step 2 (+ 인구): {merged_df.shape}")

# 병합 후 검증
print(f"\n✅ 데이터 병합 완료")
print(f"   최종 데이터: {merged_df.shape[0]}개 자치구, {merged_df.shape[1]}개 컬럼")

# 결측치 확인
null_count = merged_df.isnull().sum().sum()
if null_count > 0:
    print(f"\n⚠️ 결측치 발견: {null_count}개")
    print(merged_df.isnull().sum()[merged_df.isnull().sum() > 0])
else:
    print(f"\n✅ 결측치 없음")

In [ ]:
# 병합된 데이터 기본 정보
print("\n📊 통합 데이터 기본 정보:")
print(f"   자치구 수: {len(merged_df)}")
print(f"   총 컬럼 수: {len(merged_df.columns)}")
print(f"\n컬럼 목록:")
for i, col in enumerate(merged_df.columns, 1):
    print(f"   {i:2d}. {col}")

# 처음 3개 행 확인
print(f"\n처음 3개 행:")
merged_df.head(3)

## 3. 파생변수 생성

### 3.1 인구당 CCTV 지표

In [ ]:
# 인구당 CCTV 지표 생성 (인구 1000명당)
print("🔢 인구당 CCTV 지표 생성 중...\n")

# 전체 CCTV
merged_df['인구당_총CCTV'] = (merged_df['총_CCTV'] / merged_df['인구수'] * 1000).round(2)

# 유형별 CCTV (인구 1000명당)
for cctv_type in CCTV_RANGE.keys():
    col_name = f'인구당_{cctv_type}'
    merged_df[col_name] = (merged_df[cctv_type] / merged_df['인구수'] * 1000).round(2)

# 면적당 CCTV (선택)
merged_df['면적당_총CCTV'] = (merged_df['총_CCTV'] / merged_df['면적_km2']).round(2)

print("✅ 인구당 CCTV 지표 생성 완료")
print(f"   생성된 지표: {len(CCTV_RANGE) + 2}개")
print(f"   - 인구당_총CCTV")
for cctv_type in CCTV_RANGE.keys():
    print(f"   - 인구당_{cctv_type}")
print(f"   - 면적당_총CCTV")

### 3.2 인구당 범죄율 지표

In [ ]:
# 인구당 범죄율 지표 생성 (인구 1000명당)
print("\n🔢 인구당 범죄율 지표 생성 중...\n")

# CCTV 효과 범죄율 (주 분석 지표)
merged_df['인구당_CCTV효과범죄율'] = (
    merged_df['CCTV효과범죄_합계'] / merged_df['인구수'] * 1000
).round(2)

# 총 범죄율
merged_df['인구당_총범죄율'] = (
    merged_df['총_범죄'] / merged_df['인구수'] * 1000
).round(2)

# 범죄 유형별 비율 (인구 1000명당)
for crime_type in CRIME_RANGE.keys():
    col_name = f'인구당_{crime_type}율'
    merged_df[col_name] = (merged_df[crime_type] / merged_df['인구수'] * 1000).round(2)

print("✅ 인구당 범죄율 지표 생성 완료")
print(f"   생성된 지표: {len(CRIME_RANGE) + 2}개")
print(f"   - 인구당_CCTV효과범죄율 (주 분석 지표)")
print(f"   - 인구당_총범죄율")
for crime_type in CRIME_RANGE.keys():
    print(f"   - 인구당_{crime_type}율")

### 3.3 추가 파생변수

In [ ]:
# 추가 파생변수 생성
print("\n🔢 추가 파생변수 생성 중...\n")

# 1. 방범용 CCTV 비중 (전체 대비)
merged_df['방범용CCTV_비중'] = (
    merged_df['방범용'] / merged_df['총_CCTV'] * 100
).round(2)

# 2. CCTV 효과 범죄 비중 (전체 범죄 대비)
merged_df['CCTV효과범죄_비중'] = (
    merged_df['CCTV효과범죄_합계'] / merged_df['총_범죄'] * 100
).round(2)

# 3. CCTV 밀도 등급 (사분위수 기준)
cctv_quartiles = merged_df['인구당_총CCTV'].quantile([0.25, 0.5, 0.75])
def classify_cctv_density(value):
    if value <= cctv_quartiles[0.25]:
        return '낮음'
    elif value <= cctv_quartiles[0.5]:
        return '보통'
    elif value <= cctv_quartiles[0.75]:
        return '높음'
    else:
        return '매우높음'

merged_df['CCTV밀도_등급'] = merged_df['인구당_총CCTV'].apply(classify_cctv_density)

# 4. 범죄율 등급 (사분위수 기준)
crime_quartiles = merged_df['인구당_CCTV효과범죄율'].quantile([0.25, 0.5, 0.75])
def classify_crime_rate(value):
    if value <= crime_quartiles[0.25]:
        return '낮음'
    elif value <= crime_quartiles[0.5]:
        return '보통'
    elif value <= crime_quartiles[0.75]:
        return '높음'
    else:
        return '매우높음'

merged_df['범죄율_등급'] = merged_df['인구당_CCTV효과범죄율'].apply(classify_crime_rate)

print("✅ 추가 파생변수 생성 완료")
print(f"   - 방범용CCTV_비중")
print(f"   - CCTV효과범죄_비중")
print(f"   - CCTV밀도_등급 (낮음/보통/높음/매우높음)")
print(f"   - 범죄율_등급 (낮음/보통/높음/매우높음)")

## 4. 통합 데이터 요약 통계

In [ ]:
# 주요 파생변수 요약 통계
print("\n📊 주요 파생변수 요약 통계:\n")

key_metrics = [
    '인구당_총CCTV',
    '인구당_방범용',
    '인구당_CCTV효과범죄율',
    '인구당_총범죄율',
    '방범용CCTV_비중',
    'CCTV효과범죄_비중'
]

summary_stats = create_summary_stats(merged_df, key_metrics)
display(summary_stats)

In [ ]:
# 등급별 분포 확인
print("\n📊 CCTV 밀도 등급 분포:")
print(merged_df['CCTV밀도_등급'].value_counts().sort_index())

print("\n📊 범죄율 등급 분포:")
print(merged_df['범죄율_등급'].value_counts().sort_index())

## 5. 상위/하위 자치구 분석

In [ ]:
# 인구당 CCTV 상위/하위 5개 자치구
print("\n🏆 인구당 총CCTV 상위 5개 자치구:\n")
top5_cctv = merged_df.nlargest(5, '인구당_총CCTV')[
    ['자치구', '인구당_총CCTV', '인구당_방범용', '총_CCTV', '인구수']
]
display(top5_cctv)

print("\n📉 인구당 총CCTV 하위 5개 자치구:\n")
bottom5_cctv = merged_df.nsmallest(5, '인구당_총CCTV')[
    ['자치구', '인구당_총CCTV', '인구당_방범용', '총_CCTV', '인구수']
]
display(bottom5_cctv)

In [ ]:
# 인구당 범죄율 상위/하위 5개 자치구
print("\n🏆 인구당 CCTV효과범죄율 상위 5개 자치구 (범죄 많음):\n")
top5_crime = merged_df.nlargest(5, '인구당_CCTV효과범죄율')[
    ['자치구', '인구당_CCTV효과범죄율', 'CCTV효과범죄_합계', '인구수', '인구당_총CCTV']
]
display(top5_crime)

print("\n📉 인구당 CCTV효과범죄율 하위 5개 자치구 (범죄 적음):\n")
bottom5_crime = merged_df.nsmallest(5, '인구당_CCTV효과범죄율')[
    ['자치구', '인구당_CCTV효과범죄율', 'CCTV효과범죄_합계', '인구수', '인구당_총CCTV']
]
display(bottom5_crime)

## 6. 간단한 시각화

In [ ]:
# 인구당 CCTV vs 인구당 범죄율 산점도 (초기 탐색)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 좌측: 전체 CCTV vs 범죄율
axes[0].scatter(merged_df['인구당_총CCTV'], merged_df['인구당_CCTV효과범죄율'],
                alpha=0.6, s=100, edgecolors='black')
axes[0].set_xlabel('인구당 총CCTV (대/천명)', fontsize=12)
axes[0].set_ylabel('인구당 CCTV효과범죄율 (건/천명)', fontsize=12)
axes[0].set_title('인구당 CCTV vs 범죄율', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)

# 우측: 방범용 CCTV vs 범죄율
axes[1].scatter(merged_df['인구당_방범용'], merged_df['인구당_CCTV효과범죄율'],
                alpha=0.6, s=100, edgecolors='black', color='green')
axes[1].set_xlabel('인구당 방범용CCTV (대/천명)', fontsize=12)
axes[1].set_ylabel('인구당 CCTV효과범죄율 (건/천명)', fontsize=12)
axes[1].set_title('인구당 방범용CCTV vs 범죄율', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
save_path = os.path.join(DATA_PATHS['figures'], 'day3_cctv_vs_crime_scatter.png')
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"✅ 그래프 저장: {save_path}")
plt.show()

# 상관계수 계산
corr_total = merged_df['인구당_총CCTV'].corr(merged_df['인구당_CCTV효과범죄율'])
corr_bangbeom = merged_df['인구당_방범용'].corr(merged_df['인구당_CCTV효과범죄율'])

print(f"\n📊 상관계수:")
print(f"   인구당 총CCTV vs 범죄율: {corr_total:.4f}")
print(f"   인구당 방범용CCTV vs 범죄율: {corr_bangbeom:.4f}")

## 7. 통합 데이터 저장

In [ ]:
# 최종 통합 데이터 저장
print("\n💾 통합 데이터 저장 중...\n")

integrated_path = os.path.join(DATA_PATHS['processed'], 'integrated_data.csv')
save_csv_safely(merged_df, integrated_path, index=False, encoding='utf-8-sig')

print(f"\n✅ 통합 데이터 저장 완료")
print(f"   파일: {integrated_path}")
print(f"   크기: {merged_df.shape[0]}행 × {merged_df.shape[1]}열")
print(f"   자치구 수: {len(merged_df)}개")
print(f"   파생변수 포함 총 컬럼 수: {len(merged_df.columns)}개")

In [ ]:
# 최종 컬럼 리스트 출력
print("\n📋 최종 통합 데이터 컬럼 리스트:\n")

# 컬럼을 카테고리별로 분류
original_cols = ['자치구']
cctv_cols = [col for col in merged_df.columns if any(x in col for x in ['CCTV', 'cctv', '방범용', '교통단속용', '어린이안전용', '기타'])]
crime_cols = [col for col in merged_df.columns if any(x in col for x in ['범죄', '절도', '강도', '차량', '폭력', '성범죄'])]
population_cols = [col for col in merged_df.columns if any(x in col for x in ['인구', '면적', '밀도'])]
derived_cols = [col for col in merged_df.columns if '인구당' in col or '비중' in col or '등급' in col]

print(f"[기본 정보] ({len(original_cols)}개)")
for col in original_cols:
    print(f"  - {col}")

print(f"\n[CCTV 관련] ({len([c for c in cctv_cols if c not in derived_cols])}개)")
for col in cctv_cols:
    if col not in derived_cols and col != '자치구':
        print(f"  - {col}")

print(f"\n[범죄 관련] ({len([c for c in crime_cols if c not in derived_cols])}개)")
for col in crime_cols:
    if col not in derived_cols:
        print(f"  - {col}")

print(f"\n[인구 관련] ({len([c for c in population_cols if c not in derived_cols])}개)")
for col in population_cols:
    if col not in derived_cols:
        print(f"  - {col}")

print(f"\n[파생 변수] ({len(derived_cols)}개)")
for col in derived_cols:
    print(f"  - {col}")

## 8. Day 3 요약 및 다음 단계

In [ ]:
# Day 3 요약 리포트
summary = f"""
{'='*70}
Day 3 데이터 통합 및 파생변수 생성 요약
{'='*70}

1. 데이터 병합
   ✅ CCTV + 범죄 + 인구 데이터 병합 완료
   ✅ 자치구 수: {len(merged_df)}개
   ✅ 결측치: {'없음' if merged_df.isnull().sum().sum() == 0 else merged_df.isnull().sum().sum()}

2. 생성된 파생변수
   - 인구당 CCTV 지표: {len(CCTV_RANGE) + 2}개
   - 인구당 범죄율 지표: {len(CRIME_RANGE) + 2}개
   - 추가 지표: 4개 (비중, 등급)
   - 총 파생변수: {len(derived_cols)}개

3. 주요 통계
   - 평균 인구당 총CCTV: {merged_df['인구당_총CCTV'].mean():.2f} 대/천명
   - 평균 인구당 방범용CCTV: {merged_df['인구당_방범용'].mean():.2f} 대/천명
   - 평균 인구당 CCTV효과범죄율: {merged_df['인구당_CCTV효과범죄율'].mean():.2f} 건/천명
   - 평균 방범용CCTV 비중: {merged_df['방범용CCTV_비중'].mean():.2f}%

4. 초기 상관분석
   - 인구당 총CCTV vs 범죄율: {corr_total:.4f}
   - 인구당 방범용CCTV vs 범죄율: {corr_bangbeom:.4f}

5. 최종 데이터
   ✅ 통합 데이터 저장 완료
   ✅ 크기: {merged_df.shape[0]}행 × {merged_df.shape[1]}열
   ✅ 파일: integrated_data.csv

6. 다음 단계 (Day 4 - 2025-07-07)
   - 상관분석 (Pearson, Spearman)
   - 기초 시각화 (Bar, Scatter, Heatmap)
   - 자치구별 비교 분석
   - 상위/하위 자치구 특성 분석

{'='*70}
"""

print(summary)

# 요약 파일 저장
summary_path = os.path.join(DATA_PATHS['reports'], 'day3_summary.txt')
with open(summary_path, 'w', encoding='utf-8') as f:
    f.write(summary)
print(f"✅ 요약 파일 저장: {summary_path}")

print("\n" + "="*70)
print("🎉 Day 3 작업 완료!")
print("="*70)